In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')
df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


Note: No Null attributes identified on numerical features

In [4]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


In [5]:
total_amount = df['order_amount'].sum()
total_items = df['total_items'].sum()
total_count_items = df['order_amount'].count() #df['total_items'].count()

AOV_rectified = total_amount/total_items
print(AOV_rectified)

AOV_reported = total_amount/total_count_items
print(AOV_reported)

357.92152221412965
3145.128


## For First Question Part - A

#### ANSWER: 
Looking at the data above, The actual mean of `order_amount -$375.921`. According to shopify data, the naively calculated AOV was $3145.13. Instead of calculating the sum of 'total_items', employee has calculated the total count of 'total_items' - resulting in false AOV.

Looking at AOV_rectified & AOV_reported, we can clearly see the difference. Along with that, df.describe() shows the calculated mean as `3145.128000`. And the calculation behind that was clearly same as AOV_reported.

## For First Question Part-B

#### ANSWER: 
For shopping data such as Shopify, If we want to calculate the AOV of the sneaker shops - better approch can be to look at individual shops data & clearly understand the total purchases by an individual shop & follow a metrics like mean, median, mode. Later we use MMR(mode to mean ratio) if mode & mean has high difference.

In [6]:
# Before going into that let's see of the given data is actual for 30-days window
df.sort_values(by='created_at')

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
1862,1863,39,738,536,4,cash,2017-03-01 0:08:09
1741,1742,39,910,268,2,cash,2017-03-01 0:10:19
3228,3229,97,912,324,2,cash,2017-03-01 0:14:12
1267,1268,80,798,290,2,credit_card,2017-03-01 0:19:31
2689,2690,49,799,258,2,credit_card,2017-03-01 0:22:25
...,...,...,...,...,...,...,...
2765,2766,9,708,236,2,debit,2017-03-30 9:22:41
4890,4891,63,853,136,1,cash,2017-03-30 9:27:00
244,245,32,928,202,2,cash,2017-03-30 9:30:28
211,212,22,712,292,2,credit_card,2017-03-30 9:40:40


Given data is clearly for 30-day window

In [7]:
data_copy = df.copy()

In [8]:
# Calculating the total purchases amount, total items by individual shop_id
total_items_by_shopID = df.groupby(['shop_id'])['total_items'].sum()
total_amount_by_shopID = df.groupby(['shop_id'])['order_amount'].sum()
new_df = pd.concat([total_items_by_shopID,total_amount_by_shopID], axis=1).reset_index(level=0)
new_df['AOV'] = new_df['order_amount']/new_df['total_items']
new_df

,shop_id,total_items,order_amount,AOV
0,1,86,13588,158.0
1,2,102,9588,94.0
2,3,99,14652,148.0
3,4,103,13184,128.0
4,5,92,13064,142.0
...,...,...,...,...
95,96,110,16830,153.0
96,97,96,15552,162.0
97,98,107,14231,133.0
98,99,94,18330,195.0


In [9]:
new_df.describe()

,shop_id,total_items,order_amount,AOV
count,100.000000,100.000000,1.000000e+02,100.000000
mean,50.500000,439.360000,1.572564e+05,407.990000
std,29.011492,3396.366111,1.216218e+06,2557.462906
min,1.000000,67.000000,6.840000e+03,90.000000
25%,25.750000,88.000000,1.293050e+04,132.750000
50%,50.500000,100.000000,1.488750e+04,153.000000
75%,75.250000,111.250000,1.760000e+04,168.250000
max,100.000000,34063.000000,1.199018e+07,25725.000000


In [10]:
print('Mean:',new_df['AOV'].mean())
print('Median:',new_df['AOV'].median())
print('Mode:',new_df['AOV'].mode())

Mean: 407.99
Median: 153.0
Mode: 0    153.0
dtype: float64


Looking at the above metrics, mode & median is same for this data. But mean appears to be higher than median. This usual happens due to data anaomly. Let's have a look at our main df & new_df again to get a clear picture.

In [11]:
new_df.sort_values(by='order_amount',ascending=False)

,shop_id,total_items,order_amount,AOV
41,42,34063,11990176,352.0
77,78,88,2263800,25725.0
88,89,118,23128,196.0
80,81,128,22656,177.0
5,6,121,22627,187.0
...,...,...,...,...
1,2,102,9588,94.0
99,100,77,8547,111.0
55,56,69,8073,117.0
31,32,79,7979,101.0


Based on above results, `Shop_id['42']` has higher purchase history with `total items` = 3406, `order_amount` = $11990176
`shop_id['78']` has purchase history of total_items = 88 & order_amount = 2263800. And AOV of `shop_id['78']` looks unusal. Lets have a look at original data.

In [12]:
df[df['shop_id']==78] 

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57
490,491,78,936,51450,2,debit,2017-03-26 17:08:19
493,494,78,983,51450,2,cash,2017-03-16 21:39:35
511,512,78,967,51450,2,cash,2017-03-09 7:23:14
617,618,78,760,51450,2,cash,2017-03-18 11:18:42
691,692,78,878,154350,6,debit,2017-03-27 22:51:43
1056,1057,78,800,25725,1,debit,2017-03-15 10:16:45
1193,1194,78,944,25725,1,debit,2017-03-16 16:38:26
1204,1205,78,970,25725,1,credit_card,2017-03-17 22:32:21
1259,1260,78,775,77175,3,credit_card,2017-03-27 9:27:20


In [13]:
df[df['shop_id']==42] 

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
40,41,42,793,352,1,credit_card,2017-03-24 14:15:41
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
308,309,42,770,352,1,credit_card,2017-03-11 18:14:39
409,410,42,904,704,2,credit_card,2017-03-04 14:32:58
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
834,835,42,792,352,1,cash,2017-03-25 21:31:25
835,836,42,819,704,2,cash,2017-03-09 14:15:15
938,939,42,808,1056,3,credit_card,2017-03-13 23:43:45
979,980,42,744,352,1,debit,2017-03-12 13:09:04


By comparing the both `shop_id[42,78]`, the individual item cost of each order appears to be constant. Assume that each shop sells only one model, the prices of these models may change. And `shop_id['42']` appears to be selling high-end model & price of a shoe is `$352` - Looks geniune. 

Both for `shop_id['78']`, the individual item cost of each order is `$25725`. And this appears to be a mistake in data entry or a fraudulent data. Due to unusual pattern of this data, we cannot use this entry_point for AOV calculation till we get a standard report. As a result, by assuming this data as an anaomly - we're dropping `shop_id['78']` data.

In [14]:
#Remove data anaomly
new_df.drop(index=77, inplace=True)

In [15]:
new_df

,shop_id,total_items,order_amount,AOV
0,1,86,13588,158.0
1,2,102,9588,94.0
2,3,99,14652,148.0
3,4,103,13184,128.0
4,5,92,13064,142.0
...,...,...,...,...
95,96,110,16830,153.0
96,97,96,15552,162.0
97,98,107,14231,133.0
98,99,94,18330,195.0


In [17]:
Mean = new_df['AOV'].mean()
Median = new_df['AOV'].median()
Mode = new_df['AOV'].mode()
print('Mean:',Mean)
print('Median:',Median)
print('Mode:',Mode)

Mean: 152.26262626262627
Median: 153.0
Mode: 0    153.0
dtype: float64


In [19]:
MMR = Mode/Mean*100
MMR

# If MMR is higher, mean is our AOV & if MMR is lower than 67%, we can use mode as our AOV.

0    100.484278
dtype: float64

## For First Question Part-C

### Answer
By eliminating the anamoly from our data, mode & median appear to be same. As a result, we conclude that 50% of customers spent equal or less than & other 50% of cutomers spent equal or more than `$153.00`. And based on Mode to mean ratio calculation, our AOV is `$152.2`.

# Second Question

Question 2: For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

### 1.How many orders were shipped by Speedy Express in total?
	Ans: 54

`   SELECT COUNT(DISTINCT orderID) as total_orders
    FROM Orders
    JOIN Shippers
    ON Orders.ShipperID = Shippers.ShipperID
    WHERE Shippers.ShipperName = 'Speedy Express' `


### 2.What is the last name of the employee with the most orders?

Ans: Peacock, 40

`    SELECT e.LastName, COUNT(DISTINCT o.OrderID) as total_count
    FROM Orders o
    JOIN Employees e
    ON o.EmployeeID = e.employeeID
    GROUP BY o.EmployeeID
    ORDER BY total_count desc
    LIMIT 1 `





### 3.What product was ordered the most by customers in Germany?

Ans: ‘Boston Crab Meat’ orders: 4, 160 (In-terms of quantity)

‘Gorgonzola Telino’, 5, 125 (In-terms of total distinct orders) 

`   SELECT p.ProductName, COUNT(DISTINCT od.OrderID) as orders, SUM(od.Quantity) as quantity
    FROM Orders o
    JOIN OrderDetails od
    ON od.OrderID = o.OrderID
    JOIN Customers c
    ON o.CustomerID = c.CustomerID
    JOIN Products p
    ON od.ProductID = p.ProductID
    WHERE c.Country = 'Germany'
    GROUP BY p.ProductID
    ORDER BY quantity desc  `
